# 3.4 LangChain을 활용한 Vector Database 변경 (Chroma -> Pinecone+ upstage)
- 임베딩 길이를 조정해줘야 함

<div style="text-align: right"> Initial issue : 2022.09.25 </div>
<div style="text-align: right"> last update : 2022.09.25 </div>

할일 정리   
1. 문서의 내용을 읽는다.  
2. 문서를 쪼갠다.  
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면(입력이 길면) 답변을 생성하지 못할 수 있다. 오래걸린다.  
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

여기서는 문서 내용을 읽기 위해 Docx2txtLoader를 사용한다.  
- https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/

- https://python.langchain.com/docs/integrations/vectorstores/pinecone/

- https://app.pinecone.io/

- https://developers.upstage.ai/docs/apis/embeddings

업스테이지 임베딩의 사이즈는 4096이다. 

```
embedding array
The embedding vector is a sequence of floating-point numbers. Currently, the models utilize vectors with a dimensionality of 4096.
```

- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
우리가 자주 사용하는 Open AI의 text-embedding-3-large 임베딩의 사이즈는 3072 이다.   

```
By default, the length of the embedding vector will be 1536 for text-embedding-3-small or 3072 for text-embedding-3-large. You can reduce the dimensions of the embedding by passing in the dimensions parameter without the embedding losing its concept-representing properties. We go into more detail on embedding dimensions in the embedding use case section.
```

인덱스를 다시 만들면 된다.

### 1. 패키지 설치

In [1]:
# %pip install --upgrade --quiet  docx2txt langchain-community==0.2.16
%pip install --upgrade --quiet langchain-pinecone==0.1.3 langchain-community==0.2.16

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
load_dotenv()

True

# 2. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [7]:
len(document_list)

220

In [8]:
# # 임베딩 진행
# from langchain_openai import OpenAIEmbeddings

# # OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
# embedding = OpenAIEmbeddings(model='text-embedding-3-large') # 디펄트 대신 성능 좋은 것으로...

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings


load_dotenv()
embedding = UpstageEmbeddings(model = "solar-embedding-1-large")

In [9]:
# import os
# pinecone_api_key = os.environ.get("PINECONE_API_KEY")

# from pinecone import Pinecone

# pc = Pinecone(api_key=pinecone_api_key)

In [10]:
#%pip install -q langchain-chroma

In [11]:
# from langchain_chroma import Chroma

# # 데이터를 처음 저장할 때 
# # database = Chroma.from_documents(
# #     documents=document_list, 
# #     embedding=embedding, 
# #     collection_name='chroma-tax',# 테이블 이름
# #     persist_directory="./chroma" # 임베딩 결과를 저장할 경로
# #     )

# # 이미 저장된 데이터를 사용할 때 
# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [12]:
import os 
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "tax-upstage-index"
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name,
)


/Users/sguys99/anaconda3/envs/rag/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [13]:
# 테스트
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=5)

In [14]:
retrieved_docs

[Document(metadata={'source': './tax.docx'}, page_content='1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “공제대상자녀”라 한다) 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다. <개정 2015. 5. 13., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2022. 12. 31., 2023. 12. 31.>\n\n1. 1명인 경우: 연 15만원\n\n2. 2명인 경우: 연 35만원\n\n3. 3명 이상인 경우: 연 35만원과 2명을 초과하는 1명당 연 30만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [15]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [16]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [17]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [18]:
print(prompt[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [20]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]), llm=ChatUpstage(client=<openai.resources.chat.completions.Completions object at 0x13a248690>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x165e11990>, upstage_api_key=SecretStr('**********'))), document_variable_name='context'), retriever=VectorStoreRetriever(tags=[

In [21]:
ai_message = qa_chain({"query": query})

/var/folders/l_/8kygc87n5q1d13jbjvmg1mdm0000gn/T/ipykernel_41071/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query": query})


In [22]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [23]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉이 5천만원인 직장인의 소득세는 1,190,000원입니다.'}

- ? 프롬프트에 query 자리가 없는데 입력되는 이유?